In [24]:
import pandas as pd
import modules.translator as trans
import modules.helper as helper

df = pd.DataFrame()

for year in range(18, 23):
    tmp = pd.read_csv(f"./sources/data/squads/players_{year}.csv",
                     low_memory=False)
    tmp['season'] = f"20{year - 1}-20{year}"
    df = df.append(tmp)

competitions = [
    "Spain Primera Division",
    "German 1. Bundesliga",
    "English Premier League",
    "French Ligue 1",
    "Italian Serie A",
]
df = df.loc[df["league_name"].isin(competitions)]

to_drop = [
    "sofifa_id",
    "player_url",
    "long_name",
    "dob",
    "player_face_url",
    "club_logo_url",
    "club_flag_url",
    "nation_logo_url",
    "nation_flag_url",
    "club_team_id",
    "league_level",
    "club_position",
    "club_jersey_number",
    "club_loaned_from",
    "league_name",
    "nationality_id",
    "nationality_name",
    "nation_team_id",
    "nation_position",
    "nation_jersey_number",
    "preferred_foot",
    "weak_foot",
    "skill_moves",
    "body_type",
    "real_face",
    "release_clause_eur",
    "player_tags",
    "player_traits",
    "ls",
    "st",
    "rs",
    "lw",
    "lf",
    "cf",
    "rf",
    "rw",
    "lam",
    "cam",
    "ram",
    "lm",
    "lcm",
    "cm",
    "rcm",
    "rm",
    "lwb",
    "ldm",
    "cdm",
    "rdm",
    "rwb",
    "lb",
    "lcb",
    "cb",
    "rcb",
    "rb",
    "gk",
]
df.drop(to_drop, axis=1, inplace=True)


df["club_name"].replace(
    helper.invert_dictionary(trans.fifa_translations()),
    inplace=True,
)

In [25]:
def get_top_n(df, n):
    res = pd.DataFrame()
    for team in df['club_name'].unique():
        for season in [f"{x}-{x+1}" for x in range(2017, 2023)]:
            tmp = df.loc[(df['club_name'] == team) & (df['season'] == season)]
            tmp = tmp.nlargest(n=n, columns=['value_eur', 'wage_eur'])
            res = res.append(tmp)
    res.reset_index(drop=True, inplace=True)
    return res

players_top_15 = get_top_n(df.loc[df['player_positions'] != "GK"], 15)
goalies_top_1 = get_top_n(df.loc[df['player_positions'] == "GK"], 1)

In [26]:
##### squad
squad = players_top_15.append(goalies_top_1)
squad = squad.groupby(['club_name', 'season']).mean().round(2)

common_features = ['overall',
                   'potential',
                   'value_eur',
                   'wage_eur',
                   'age',
                   'height_cm',
                   'weight_kg',
                   'club_contract_valid_until',
                   'international_reputation']
squad = squad.loc[:, common_features]

# squad.reset_index(inplace=True)

In [27]:
def drop_startswith(df, prefixes):
    for prefix in prefixes:
        df = df.loc[:, ~df.columns.str.startswith(prefix)]
    return df

In [28]:
##### players
players = players_top_15
players = players.groupby(['club_name', 'season']).mean().round(2)

prefixes = ['goalkeeping']
players = drop_startswith(players, prefixes)
players.drop(common_features, axis=1, inplace=True, errors='ignore')

# players.reset_index(inplace=True)

In [29]:
##### goalies
goalies = goalies_top_1
goalies = goalies.groupby(['club_name', 'season']).mean().round(2)

prefixes = ['attacking',
            'skill',
            'movement',
            'power',
            'mentality',
            'defending']
goalies = drop_startswith(goalies, prefixes)
goalies.dropna(axis=1, inplace=True)
goalies.drop(common_features, axis=1, inplace=True, errors='ignore')

# goalies.reset_index(inplace=True)

In [30]:
result = squad.join(players).join(goalies).reset_index()
# import modules.visualizer as vis
# vis.info_me(df)
# vis.describe_me(df)
# vis.corr_me(df)

In [32]:
result.to_csv("./sources/data/squads.csv")